In [10]:
import pandas as pd
import numpy as np

wd = '/Users/finn/Documents/GitHub/data/e4e3/'

In [11]:
df = pd.read_pickle(f"{wd}/Uncleaned Data/LAD_incomes.pkl")
df = pd.melt(df, id_vars= ['Region', 'LAD code', 'Region name'])
df.columns = ['Region', 'LAD code', 'Region name', 'Year', 'Nominal_GDHI']
df

,Region,LAD code,Region name,Year,Nominal_GDHI
0,North East,E06000001,Hartlepool,1997,8398.0
1,North East,E06000004,Stockton-on-Tees,1997,9756.0
2,North East,E06000002,Middlesbrough,1997,8654.0
3,North East,E06000003,Redcar and Cleveland,1997,9363.0
4,North East,E06000005,Darlington,1997,9638.0
...,...,...,...,...,...
8597,North West,E08000011,Knowsley,2019,16627.0
8598,North West,E08000013,St. Helens,2019,17325.0
8599,North West,E08000012,Liverpool,2019,15673.0
8600,North West,E08000014,Sefton,2019,19325.0


Convert to 2020 prices:

In [12]:
cpi_df = pd.read_csv(f"{wd}/Uncleaned Data/uk_price_index.csv", skiprows=7)
cpi_df.columns = ['Date', "cpi_index"]

cpi_df['Year'] = cpi_df['Date'].str[0:4]
cpi_df = cpi_df.drop_duplicates(subset=['Year'], keep='last')[['Year', 'cpi_index']]
index_2020 = float(cpi_df[cpi_df['Year']=='2020']['cpi_index'])
cpi_df['cpi_index'] = (cpi_df['cpi_index']/index_2020)
cpi_index = dict(zip(list(cpi_df['Year']), list((cpi_df['cpi_index']))))

In [13]:
df

,Region,LAD code,Region name,Year,Nominal_GDHI
0,North East,E06000001,Hartlepool,1997,8398.0
1,North East,E06000004,Stockton-on-Tees,1997,9756.0
2,North East,E06000002,Middlesbrough,1997,8654.0
3,North East,E06000003,Redcar and Cleveland,1997,9363.0
4,North East,E06000005,Darlington,1997,9638.0
...,...,...,...,...,...
8597,North West,E08000011,Knowsley,2019,16627.0
8598,North West,E08000013,St. Helens,2019,17325.0
8599,North West,E08000012,Liverpool,2019,15673.0
8600,North West,E08000014,Sefton,2019,19325.0


In [14]:
df['Real_GDHI'] = df.apply(lambda x: x['Nominal_GDHI']/(cpi_index[x['Year']]), axis=1)

In [15]:
df

,Region,LAD code,Region name,Year,Nominal_GDHI,Real_GDHI
0,North East,E06000001,Hartlepool,1997,8398.0,12994.925035
1,North East,E06000004,Stockton-on-Tees,1997,9756.0,15096.271570
2,North East,E06000002,Middlesbrough,1997,8654.0,13391.055163
3,North East,E06000003,Redcar and Cleveland,1997,9363.0,14488.149929
4,North East,E06000005,Darlington,1997,9638.0,14913.680339
...,...,...,...,...,...,...
8597,North West,E08000011,Knowsley,2019,16627.0,16764.919816
8598,North West,E08000013,St. Helens,2019,17325.0,17468.709677
8599,North West,E08000012,Liverpool,2019,15673.0,15803.006452
8600,North West,E08000014,Sefton,2019,19325.0,19485.299539


In [16]:
def reindexed(x):
    initial_value = df[df['LAD code']==x['LAD code']].iloc[0,5]
    return (x['Real_GDHI']-initial_value)/initial_value

adding a column showing data, reindexed to 1997:

In [17]:
df['Change'] = df.apply(reindexed, axis=1)

In [18]:
df

,Region,LAD code,Region name,Year,Nominal_GDHI,Real_GDHI,Change
0,North East,E06000001,Hartlepool,1997,8398.0,12994.925035,0.000000
1,North East,E06000004,Stockton-on-Tees,1997,9756.0,15096.271570,0.000000
2,North East,E06000002,Middlesbrough,1997,8654.0,13391.055163,0.000000
3,North East,E06000003,Redcar and Cleveland,1997,9363.0,14488.149929,0.000000
4,North East,E06000005,Darlington,1997,9638.0,14913.680339,0.000000
...,...,...,...,...,...,...,...
8597,North West,E08000011,Knowsley,2019,16627.0,16764.919816,0.428017
8598,North West,E08000013,St. Helens,2019,17325.0,17468.709677,0.347802
8599,North West,E08000012,Liverpool,2019,15673.0,15803.006452,0.209466
8600,North West,E08000014,Sefton,2019,19325.0,19485.299539,0.104017


In [19]:
df.to_csv(f"{wd}/UK_LAD_Incomes.csv", index=False)

In [20]:
df.sort_values(by='Change', ascending=False)

,Region,LAD code,Region name,Year,Nominal_GDHI,Real_GDHI,Change
8405,London,E09000012,Hackney,2019,24575.0,24778.847926,1.309068
8406,London,E09000025,Newham,2019,20710.0,20881.788018,1.297005
8031,London,E09000012,Hackney,2018,23527.0,24032.248366,1.239495
7283,London,E09000012,Hackney,2016,22261.0,23829.289628,1.220582
6909,London,E09000012,Hackney,2015,21621.0,23559.137450,1.195407
...,...,...,...,...,...,...,...
377,North East,E06000003,Redcar and Cleveland,1998,9067.0,13776.802778,-0.049099
6454,South West,E06000053,Isles of Scilly,2014,18448.0,20202.314314,-0.056799
376,North East,E06000002,Middlesbrough,1998,8298.0,12608.350000,-0.058450
596,East of England,E06000032,Luton,1998,8851.0,13448.602778,-0.060615
